In [3]:
import os
import json
import pandas as pd
from pandas import DataFrame

def data_merge(source_path:str,second:DataFrame) -> DataFrame:
    site_2023_df = pd.read_csv(source_path)
    site_2023_df.columns = ['日期','code','進站','出站']
    merge2023 = pd.merge(site_2023_df,second)
    merge2023['日期'] = pd.to_datetime(merge2023['日期'].astype(str))
    merge2023_1 = merge2023.set_index('日期')
    merge2023_2 = merge2023_1.reindex(columns=['站名','地址','進站','出站'])
    return merge2023_2


with open('每日各站進出站人數/車站基本資料集.json',encoding='utf-8') as file:
    content:list[dict] = json.load(file)
info_df = pd.DataFrame(content)
info_df1 = info_df.reindex(columns=['stationCode','stationName','stationAddrTw'])
info_df1.columns = ['code','站名','地址']
info_df1['code'] = info_df1['code'].astype(int)


merges = []
for file_name in os.listdir('每日各站進出站人數'):
    if '每日各站進出站人數' in file_name:
        merges.append(data_merge(os.path.abspath(f'每日各站進出站人數/{file_name}'),info_df1))

pd.concat(merges)

,站名,地址,進站,出站
日期,,,,
2022-01-01,基隆,基隆市仁愛區港西街5號,8645,8526
2022-01-01,三坑,基隆市仁愛區德厚里龍安街 206 號,996,1243
2022-01-01,八堵,基隆市暖暖區八南里八堵路 142 號,1369,1711
2022-01-01,七堵,基隆市七堵區長興里東新街 2 號,3663,4535
2022-01-01,百福,基隆市七堵區堵南里明德三路 1 之 1 號,1532,1845
...,...,...,...,...
2023-12-31,瑞芳,新北市瑞芳區龍潭里明燈路三段 82 號,7916,8252
2023-12-31,海科館,基隆市中正區長潭里,164,195
2023-12-31,八斗子,新北市瑞芳區建基路2段121號隔壁 砂子里省道臺 2 線 (與新北市瑞芳區交界處、無站房),652,720


### 同學的答案 

In [2]:
import os
import pandas as pd
from pandas import DataFrame

def data_merge(source_path:str,second:DataFrame)->DataFrame:
    pop_density = pd.read_csv(source_path)
    pop_density.columns = ['統計年','區域別','年底人口數','土地面積','人口密度']
    pop_density.columns = pop_density.iloc[0].values.tolist()
    pop_1 = pd.merge(pop_density,second)
    pop_1 = pop_density.drop(index=[0])
    pop_1.dropna(inplace= True)
    return pop_1

merges= []
info_df1= pd.DataFrame(columns = ['統計年','區域別','年底人口數','土地面積','人口密度'])

for file_name in os.listdir('人口密度'):
     if 'opendata' in file_name:
         merges.append(data_merge(os.path.abspath(f'人口密度/{file_name}'),info_df1))

df2 = pd.concat(merges,ignore_index=True)
df2.set_index('區域別',inplace=True)
df2.sort_values(['區域別','統計年'],ascending=True,inplace=True)
df2

,統計年,年底人口數,土地面積,人口密度
區域別,,,,
南投縣中寮鄉,103,15566,146.6541,106
南投縣中寮鄉,106,14842,146.6541,101
南投縣中寮鄉,108,14495,146.6541,99
南投縣中寮鄉,110,14013,146.6541,96
南投縣中寮鄉,111,13841,146.6541,94
...,...,...,...,...
高雄市鼓山區,106,139259,14.7523,9440
高雄市鼓山區,108,141055,14.7523,9562
高雄市鼓山區,110,140410,14.7523,9518


In [4]:
import json
import pandas as pd
from pandas import DataFrame
import os

def value_to_int(value:str)->int:
    try:
        return int(value)
    except ValueError:
        return 0

def data_merge(sourse_path:str)->DataFrame:
    people_df=pd.read_csv(sourse_path)
    people_df.columns=people_df.iloc[0]
    people_df1=people_df.drop(index=[0]).dropna()
    people_df2=people_df1.set_index('區域別')
    people_df2['年底人口數']=people_df2['年底人口數'].map(value_to_int) 
    people_df2['人口密度']=people_df2['人口密度'].map(value_to_int)
    return  people_df2

merges=[]
for file_name in os.listdir('人口密度') :
    if 'opendata' in file_name:
        merges.append(data_merge(os.path.abspath(f'人口密度/{file_name}')))


pd.concat(merges)

,統計年,年底人口數,土地面積,人口密度
區域別,,,,
新北市板橋區,103,555914,23.1373,24027
新北市三重區,103,389325,16.317,23860
新北市中和區,103,415226,20.144,20613
新北市永和區,103,227267,5.7138,39775
新北市新莊區,103,411711,19.7383,20858
...,...,...,...,...
連江縣北竿鄉,108,2595,9.9,262
連江縣莒光鄉,108,1543,4.7,328
連江縣東引鄉,108,1352,3.8,356
